1. Загрузите датасет для классификации кошек и собак: https://drive.google.com/file/d/1YQ2PX-BZ_7uZ216qmAnx-nTCaEqozuCZ/view?usp=share_link.
2. Загрузите датасет с множеством различных классов: https://data.caltech.edu/records/mzrjq-6wc02/files/caltech-101.zip. Выберите 3 любых класса, которые вам нравятся.
3. Загрузите данные и сформируете датасет. Выполните минимум две аугментации. Соберите сверточную нейронную сеть с помощью фреймворка Tensorflow и решите задачу классификации кошек и собак. Оцените модель. Будьте готовы ответить на вопросы:
    - какие этапы предварительной обработки данных вы делаете и что происходит с данными?
    - что означают параметры, которые вы задаете?
    - какие слои у вас есть и что происходит на каждом слое?
    
<b>Обращаю внимание: </b> допускается несдача ЛР даже если все сделано правильно, но на вопросы ответы не предоставлены (либо неверны).

4. Загрузите модели InceptionV3 и VGG19 с помощью Tensorflow. Выполните transfer learning и fine-tuning этих моделей для распознавания выбранных на 2 пункте классов. В процессе подготовки сетей, разморозьте какой-либо еще слой, кроме последнего. Сравните результаты двух сетей на ваших данных (по метрике accuracy в процессе обучения).
5. Используйте реализацию многослойного персептрона из ЛР 4. Реализуйте сверточный слой (прямое и обратное распространение). Соберите сверточную сеть. Попробуйте обучить классификатор кошек и собак с использованием собственной реализации сверточной нейронной сети. Вам также потребуется реализовать слой для преобразования многомерных массивов данных в одномерные.

In [6]:
import sys
sys.path.insert(0, "C://Users//MSI//Desktop//OmSTU//MachineLearning//models//Neyron")
from multilayer_perceptron import MultilayerPerceptronModel
from image_preproccessing import tf_to_ndarray, to_1D_array

import tensorflow as tf
import numpy as np
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg19 import VGG19

In [7]:
train_data3, test_data3 = tf.keras.utils.image_dataset_from_directory(
    "../data/data3",
    validation_split=0.1,
    subset="both",
    seed=42,
    image_size=(128,128),
    batch_size=16
)

train_data4, test_data4 = tf.keras.utils.image_dataset_from_directory(
    "../data/data4",
    validation_split=0.1,
    subset="both",
    seed=42,
    image_size=(128, 128),
    batch_size=16
)

Found 2000 files belonging to 2 classes.
Using 1800 files for training.
Using 200 files for validation.
Found 196 files belonging to 3 classes.
Using 177 files for training.
Using 19 files for validation.


In [8]:
data_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.2),
    ]
)
augmentation = lambda img, label: (data_augmentation(img), label)

train_data3 = train_data3.map(augmentation)
train_data3 = train_data3.map(augmentation)
train_data4 = train_data4.map(augmentation)
train_data4 = train_data4.map(augmentation)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [9]:
train_x, train_y = tf_to_ndarray(train_data3)
test_x, test_y = tf_to_ndarray(test_data3)

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

train_x = to_1D_array(train_x)
test_x = to_1D_array(test_x)

print(train_x.shape)
print(test_x.shape)

(1800, 128, 128, 3)
(1, 1800)
(200, 128, 128, 3)
(1, 200)
(49152, 1800)
(49152, 200)


In [12]:
# входной слой - размер (200, 400, 3)
inputs = tf.keras.Input(shape=(300, 300, 3))
# преобразуем значения пикселей из [0, 255] к [0, 1]
x = tf.keras.layers.Rescaling(1.0 / 255)(inputs)
# первый сверточный слой
x = tf.keras.layers.Conv2D(32, 3, strides=2, padding="same")(x)
# первый пулинг
x = tf.keras.layers.MaxPooling2D(3, strides=2, padding="same")(x)
# второй сверточный слой
x = tf.keras.layers.Conv2D(128, 4, strides=2, padding="valid")(x)
# второй пулинг
x = tf.keras.layers.MaxPooling2D(3, strides=2, padding="same")(x)
# выпрямляем многомерный массив
x = tf.keras.layers.Flatten()(x)
# полносвязный слой с 64 нейронами
x = tf.keras.layers.Dense(32, activation="tanh")(x)
# выходной слой с функцией sofrmax
outputs = tf.keras.layers.Dense(2, activation="softmax")(x)
model = tf.keras.Model(inputs, outputs)

In [13]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 300, 300, 3)]     0         
                                                                 
 rescaling_1 (Rescaling)     (None, 300, 300, 3)       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 150, 150, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 75, 75, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 36, 36, 128)       65664     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 18, 18, 128)      0         
 2D)                                                       

In [14]:
# компилируем сеть, указываем, что будем при обучении смотреть значения accuracy
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [16]:
# начинаем обучение
# указываем validation_data - при обучении будем получать accuracy для тестовой выборки
model.fit(train_data3, epochs=5, validation_data=test_data3)

Epoch 1/5
113/113 [==============================] - 43s 369ms/step - loss: 0.7185 - accuracy: 0.5572 - val_loss: 0.6968 - val_accuracy: 0.5700
Epoch 2/5
113/113 [==============================] - 38s 333ms/step - loss: 0.6607 - accuracy: 0.6011 - val_loss: 0.6218 - val_accuracy: 0.6100
Epoch 3/5
113/113 [==============================] - 38s 333ms/step - loss: 0.6291 - accuracy: 0.6383 - val_loss: 0.5900 - val_accuracy: 0.6700
Epoch 4/5
113/113 [==============================] - 39s 348ms/step - loss: 0.6237 - accuracy: 0.6500 - val_loss: 0.5891 - val_accuracy: 0.6950
Epoch 5/5
113/113 [==============================] - 40s 349ms/step - loss: 0.6083 - accuracy: 0.6711 - val_loss: 0.5625 - val_accuracy: 0.7250


In [17]:
img = tf.keras.preprocessing.image.load_img(
    "../data/data3/dogs/4.jpg", target_size=(300, 300)
)
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

predictions = model.predict(img_array)
predictions

1/1 [==============================] - 0s 113ms/step


array([[0.2622296 , 0.73777044]], dtype=float32)